In [121]:
import pandas as pd
import xarray as xr
import numpy as np
import seaborn as sns
import os
import folium
import folium.plugins as plugins
from folium.features import DivIcon

%store -r file_name

met_data = pd.read_csv(file_name)
station_data = pd.read_csv('station_info.csv').rename(columns={'id':'station'})

merged_data = pd.merge(met_data, station_data, on='station')

merged_data

# extract = merged_data[merged_data['name'].str.contains(r'Mariazell|Lilienfeld|St.P')][['tl','time','name', 'lat', 'lon']]
extract = merged_data[['tl','tlmax','time','name', 'lat', 'lon']]


# # Plot the responses for different events and regions
# sns.lineplot(x="time", y="tl",
#              hue="name",
#              data=extract)

# sns.despine()



In [122]:
extract.loc[extract['time'] == extract['time'].iloc[-1]]




,tl,tlmax,time,name,lat,lon
59166,17.2,17.7,2024-08-26T19:00+00:00,Aflenz,47.54594,15.24069
59167,16.8,16.9,2024-08-26T19:00+00:00,Aigen im Ennstal,47.53278,14.13826
59168,17.3,17.3,2024-08-26T19:00+00:00,Allentsteig,48.69083,15.36694
59169,19.1,19.2,2024-08-26T19:00+00:00,Amstetten,48.10889,14.89500
59170,15.3,15.3,2024-08-26T19:00+00:00,Bad Aussee,47.61050,13.75844
...,...,...,...,...,...,...
59680,NaN,NaN,2024-08-26T19:00+00:00,Bad Radkersburg,46.68667,15.97861
59681,NaN,NaN,2024-08-26T19:00+00:00,Bad Radkersburg,46.68583,15.97944
59682,23.5,23.7,2024-08-26T19:00+00:00,Bad Radkersburg,46.69222,15.99333
59683,18.0,18.0,2024-08-26T19:00+00:00,Loibl Tunnel,46.44389,14.24917


In [123]:
# noe_values = extract.loc[extract['time'] == extract['time'][-1]]
latest = extract.loc[extract['time'] == extract['time'].iloc[-1]]

# iterate through each row and select
# 'Name' and 'Stream' column respectively.

for ind in latest.index:
    # print('Lat: {}, Lon: {}'.format(noe_values['lat'][ind], noe_values['lon'][ind]))
    lats = [latest['lat'][ind] for ind in latest.index]
    lons = [latest['lon'][ind] for ind in latest.index]
    temp = [latest['tl'][ind] for ind in latest.index]
    tlmax = [latest['tlmax'][ind] for ind in latest.index]

type(temp[0])

numpy.float64

In [128]:
# base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
# folium.TileLayer(location=[lats[0], lons[0]], zoom_start=7,
#                 tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=MglwGpuT1VUaVc2LSRX042X3krQz6y2cOZfGtNqWg3VMUke6gemUIWz7G1uxPPXP',
#                 attr='CartoDB.Voyager', max_zoom=20).add_to(base_map)
# base_map.add_to(map)

m = folium.Map(location=[lats[0], lons[0]], zoom_start=7,
               tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=MglwGpuT1VUaVc2LSRX042X3krQz6y2cOZfGtNqWg3VMUke6gemUIWz7G1uxPPXP',
               attr='CartoDB.Voyager', max_zoom=20)

COLORS = [
    (lambda x: x > 40, "#EC8989"),
    (lambda x: 35 < x <= 40, "#EA3D09"),
    (lambda x: 30 < x <= 35, "#EA3D09"),
    (lambda x: 25 < x <= 30, "#EA6509"),
    (lambda x: 20 < x <= 25, "#C5C71B"),
    (lambda x: 15 < x <= 20, "#8EF700"),
    (lambda x: 10 < x <= 15, "#3B8701"),
    (lambda x: 5 < x <= 10, "#94EA7F"),
    (lambda x: 0 < x <= 5, "#9cecc8"),
    (lambda x: 0 >= x >= -5, "#18B5DA"),
    (lambda x: -5 > x >= -10, "#1661e8"),
    (lambda x: -10 > x >= -15, "#A375E6"),
    (lambda x: -15 > x >= -20, "#ce8bf0"),
    (lambda x: x < -20, "#EC9CE6"),
]

def get_color(x):
    for predicate, color in COLORS:
        if predicate(x):
            return color

    return 'black'

fg = folium.FeatureGroup(name="Temperature", control=True, show=True).add_to(m)

for i in range(len(lats)):
        color = get_color(temp[i])
        html_str = '<div style="font-size: 18pt; color : ' + color +'">' + str(temp[i]) + '</div>'
        p1 = [lats[i], lons[i]]
        folium.Marker(p1, icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fg)
        # m.add_child(folium.CircleMarker(p1, radius=15))
        
fgg = folium.FeatureGroup(name="Tmax", control=True, show=False).add_to(m)

for i in range(len(lats)):
        color = get_color(tlmax[i])
        html_str = '<div style="font-size: 18pt; color : ' + color +'">' + str(tlmax[i]) + '</div>'
        p1 = [lats[i], lons[i]]
        folium.Marker(p1, icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fgg)
        # m.add_child(folium.CircleMarker(p1, radius=15))
folium.LayerControl().add_to(m)


plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)


# icon = folium.DivIcon(html="17.4")
# folium.Marker([47.0, 15.0], icon=icon).add_to(m)
m.save(os.path.join('simple_popups.html'))